In [1]:
!pip install accelerate bitsandbytes peft wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.1/281.1 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410

In [2]:
import os
from pathlib import Path
from PIL import Image
import random
import yaml
from huggingface_hub import snapshot_download

os.getcwd()

'/content'

In [13]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# run in shell
!git clone https://github.com/huggingface/diffusers.git
!cd diffusers && pip install -e .
!git clone https://github.com/ManuDiehn/DreamBooth.git

Cloning into 'diffusers'...
remote: Enumerating objects: 61850, done.
remote: Counting objects: 100% (1150/1150), done.
remote: Compressing objects: 100% (639/639), done.
remote: Total 61850 (delta 729), reused 742 (delta 440), pack-reused 60700
Receiving objects: 100% (61850/61850), 43.39 MiB | 12.50 MiB/s, done.
Resolving deltas: 100% (45044/45044), done.
Obtaining file:///content/diffusers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for diffusers (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.28.0.dev0-0.editable-py3-none-any.whl size=11104 sha256=ab2532dd90c8f9f545a7b00ba6a0124165165626457dbadc5704d66d8497a699
  Stored in directory: /tmp/pip-ephem-wheel-cache-d17sldlp/wheels/95/c5/3b/e1b4269f8a2584de57e75f949a185b48fc4144e9a91fc9965a
Successfully built diffusers
Cloning 

In [5]:
SESSION_NAME = "dog"
WORKSPACE = Path('DreamBooth/output').resolve()
INSTANCE_NAME = SESSION_NAME
OUTPUT_DIR = str(WORKSPACE / "models" / SESSION_NAME)
SESSION_DIR = WORKSPACE / 'Sessions' / SESSION_NAME
INSTANCE_DIR = str(SESSION_DIR / 'instance_images')
IMAGES_FOLDER = (Path('DreamBooth/input/') / SESSION_NAME).resolve()

CONFIG_DIR_ACCELERATE = str(Path('DreamBooth/config/accelerate_config.yaml').resolve())
CONFIG_DIR_TRAINING = str(Path('DreamBooth/config/training_config.yaml').resolve())
SCRIPT_DIR_DREAMBOOTH = 'diffusers/examples/dreambooth/train_dreambooth_lora_sdxl.py'

Model_Version = 'XL1.0'
MODEL_NAME = "stabilityai/stable-diffusion-xl-base-1.0"
VAE_NAME="madebyollin/sdxl-vae-fp16-fix"

Path(INSTANCE_DIR).mkdir(parents=True, exist_ok=True)

In [ ]:
IMAGES_FOLDER

PosixPath('/content/DreamBooth/input/dog')

In [6]:
OUTPUT_DIR

'/content/DreamBooth/output/models/dog'

In [7]:
snapshot_download(
    "diffusers/dog-example",
    local_dir=IMAGES_FOLDER, repo_type="dataset",
    ignore_patterns=".gitattributes",
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

alvan-nee-Id1DBHv4fbg-unsplash.jpeg:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

alvan-nee-9M0tSjb-cpA-unsplash.jpeg:   0%|          | 0.00/677k [00:00<?, ?B/s]

alvan-nee-bQaAJCbNq3g-unsplash.jpeg:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

alvan-nee-eoqnr8ikwFE-unsplash.jpeg:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

alvan-nee-brFsZ7qszSY-unsplash.jpeg:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

'/content/DreamBooth/input/dog'

In [28]:
# check if your images have the desired size and copy them
crop_size = 1024

training_steps = 50
learning_rate = 1e-4

resolution = 512
precision="fp16" #no fp16

enable_text_encoder_training = True

train_batch_size = 1
gradient_accumulation_steps = 4

save_checkpoint_every_n_steps = True
save_checkpoint_every = 500

Seed=random.randint(1, 999999)
instance_prompt = "picmd"
validation_prompt = 'A photo of sks dog in a bucket'
validation_epochs = 25


In [29]:
for filename in Path(IMAGES_FOLDER).glob('*.jpeg'):
    print(filename)
    new_path_with_file = Path(INSTANCE_DIR) / Path(filename).name
    file = Image.open(filename)
    width, height = file.size
    if file.size != (crop_size, crop_size):
        file = file.resize((crop_size, crop_size), Image.LANCZOS)
    file.save(new_path_with_file, format="JPEG")

/content/DreamBooth/input/dog/alvan-nee-Id1DBHv4fbg-unsplash.jpeg
/content/DreamBooth/input/dog/alvan-nee-brFsZ7qszSY-unsplash.jpeg
/content/DreamBooth/input/dog/alvan-nee-9M0tSjb-cpA-unsplash.jpeg
/content/DreamBooth/input/dog/alvan-nee-bQaAJCbNq3g-unsplash.jpeg
/content/DreamBooth/input/dog/alvan-nee-eoqnr8ikwFE-unsplash.jpeg


In [30]:
def create_configs(
        config_dir_accelerate,
        config_dir_training,
        model_dir,
        vae_dir,
        instance_dir,
        output_dir,
        prompt,
        validation_prompt,
        validation_epochs,
        learning_rate,
        seed,
        train_batch_size,
        gradient_accumulation_steps,
        resolution,
        precision,
        training_steps,
    ):

    config_accelerate = {
      'compute_environment': 'LOCAL_MACHINE',
      'distributed_type': 'NO',
      'num_machines': 1,
      'num_processes': 1,
      'mixed_precision': precision,
    }

    # Write the configuration to a YAML file
    with open(config_dir_accelerate, 'w') as file:
        yaml.safe_dump(config_accelerate, file, default_flow_style=False)

    config_training = {
        'gradient_accumulation_steps': gradient_accumulation_steps,
        #'gradient_checkpointing': True,
        'instance_dir': instance_dir,
        'instance_prompt': prompt,
        'learning_rate': learning_rate,
        'lr_scheduler': 'constant',
        'lr_warmup_steps': 0,
        'max_train_steps': training_steps,
        'output_dir': output_dir,
        'pretrained_model_name_or_path': model_dir,
        'pretrained_vae_model_name_or_path': vae_dir,
        'resolution': resolution,
        'seed': seed,
        'train_batch_size': train_batch_size,
        'validation_prompt': validation_prompt,
        'precision': precision,
        'validation_epochs': validation_epochs
    }

    return config_training


config_training = create_configs(
    config_dir_accelerate=CONFIG_DIR_ACCELERATE,
    config_dir_training=CONFIG_DIR_TRAINING,
    model_dir=MODEL_NAME,
    vae_dir=VAE_NAME,
    instance_dir=INSTANCE_DIR,
    output_dir=OUTPUT_DIR,
    prompt=instance_prompt,
    validation_prompt=validation_prompt,
    validation_epochs=validation_epochs,
    learning_rate=learning_rate,
    seed=Seed,
    train_batch_size=train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    resolution=resolution,
    precision=precision,
    training_steps=training_steps,
)

In [31]:
config_training

{'gradient_accumulation_steps': 4,
 'instance_dir': '/content/DreamBooth/output/Sessions/dog/instance_images',
 'instance_prompt': 'picmd',
 'learning_rate': 0.0001,
 'lr_scheduler': 'constant',
 'lr_warmup_steps': 0,
 'max_train_steps': 50,
 'output_dir': '/content/DreamBooth/output/models/dog',
 'pretrained_model_name_or_path': 'stabilityai/stable-diffusion-xl-base-1.0',
 'pretrained_vae_model_name_or_path': 'madebyollin/sdxl-vae-fp16-fix',
 'resolution': 512,
 'seed': 516811,
 'train_batch_size': 1,
 'validation_prompt': 'A photo of sks dog in a bucket',
 'precision': 'fp16',
 'validation_epochs': 25}

In [32]:
def launch_training(config_dir_accelerate, config_training, script_dir):

    command = (
        f"accelerate launch --config_file=\"{config_dir_accelerate}\" \"{script_dir}\" "
        f"--pretrained_model_name_or_path=\"{config_training['pretrained_model_name_or_path']}\" "
        f"--instance_data_dir=\"{config_training['instance_dir']}\" "
        f"--pretrained_vae_model_name_or_path=\"{config_training['pretrained_vae_model_name_or_path']}\" "
        f"--output_dir=\"{config_training['output_dir']}\" "
        f"--instance_prompt=\"{config_training['instance_prompt']}\" "
        f"--resolution={str(config_training['resolution'])} "
        f"--train_batch_size={str(config_training['train_batch_size'])} "
        f"--gradient_checkpointing "
        f"--mixed_precision=\"{config_training['precision']}\" "
        f"--gradient_accumulation_steps={str(config_training['gradient_accumulation_steps'])} "
        f"--learning_rate={str(config_training['learning_rate'])} "
        #f"--report_to=\"wandb\" "
        f"--lr_scheduler=\"{config_training['lr_scheduler']}\" "
        f"--lr_warmup_steps={str(config_training['lr_warmup_steps'])} "
        f"--max_train_steps={str(config_training['max_train_steps'])} "
        f"--validation_prompt=\"{str(config_training['validation_prompt'])}\" "
        f"--validation_epochs={str(config_training['validation_epochs'])} "
        f"--seed={str(config_training['seed'])} "
        f"--use_8bit_adam "
        f"--push_to_hub"
    )

    get_ipython().system(command)

launch_training(
    config_dir_accelerate=CONFIG_DIR_ACCELERATE,
    config_training=config_training,
    script_dir=SCRIPT_DIR_DREAMBOOTH,
)

2024-05-20 09:32:14.962731: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-20 09:32:14.962784: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-20 09:32:14.964381: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-20 09:32:15.985329: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
05/20/2024 09:32:17 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

You are using a model of type clip_text_model to instantiate a model of ty

In [20]:
from huggingface_hub.repocard import RepoCard
from diffusers import DiffusionPipeline
import torch

lora_model_id = <"lora-sdxl-dreambooth-id">
card = RepoCard.load(lora_model_id)
base_model_id = card.data.to_dict()["base_model"]

pipe = DiffusionPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
pipe.load_lora_weights(lora_model_id)
image = pipe("A picture of a sks dog in a bucket", num_inference_steps=25).images[0]
image.save("sks_dog.png")

SyntaxError: invalid syntax (<ipython-input-20-77a63eaebe1e>, line 5)